In [1]:
import re
import string
import pandas as pd
from collections import Counter
from operator import add
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist as nF
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from pyspark import SparkContext
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
sc = SparkContext.getOrCreate()

In [3]:
df= pd.read_csv('Hotel_Reviews.csv')
df = df.sample(frac=1).reset_index(drop=True)
df=df[:100000]

In [4]:
df1=pd.DataFrame()
df1['review'] = df['Negative_Review']+df['Positive_Review']
df1['rating'] = df['Reviewer_Score']
df1.rating=df1.rating.astype(int)


In [5]:
lines = sc.parallelize(df1.values)
lines.first()

array([' Room I got should be a broom room or some staff room it was the size of a single bed so small and with a view on some giant pipe horrific this room was 170  This is a terrible hotel even the location does not make up for it in front of the hotel there is some magazine after a long discussion I got a transfer to a nice hotel which was normal ',
       3], dtype=object)

In [ ]:
del df
del df1

In [6]:
train, test = lines.randomSplit([0.6, 0.4])
train_review = train.map(lambda x: x[0])
train_rating = train.map(lambda x: x[1])
test_review = test.map(lambda x: x[0])
test_rating = test.map(lambda x: x[1])

In [7]:
# Prepare the dataset to remove to remove stop words by intializing the system
def initializeSystem():
    print ("Preparing stop words.")
    stop = set(stopwords.words('english') + punctuation + ['rt', 'via', 'i\'m', 'us', 'it'])
    for x in stop:
        stopWords.append(stemmer.stem(lemmatiser.lemmatize(x, pos="v")))

# Tokenize will separate each word as tokens
def tokenize(s):
    return tokens_re.findall(s)

# This function removes non-alhanumeric characters, removes the emoticons if present and stems and lemmatize each word
def preprocess(s, lowercase=True):
    s = re.sub('[^a-zA-Z0-9\n\.]', ' ', s)
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else stemmer.stem(lemmatiser.lemmatize(token.lower(), pos="v")) for
                  token in tokens]
    return tokens

# Each row is broken down into space separated words or terms of list and sent for stemming and lemming. Here stop words are removed
def processString(string):
    # terms_all = [term for term in preprocess(string)]
    terms_stop = [term for term in preprocess(string) if term not in stopWords and len(str(term)) > 1]
    return ' '.join(terms_stop)

def convertReviews(reviews):
    convertedReviews = []
    for index, row in reviews.iterrows():
        temp = processString(str(row["review"]).lower())
        str1 = ' '.join(temp)
        temp = [str1]
        temp.append(row["rating"])
        # print temp
        convertedReviews.append(temp)
        # convertedReviews = convertedReviews + ',' + processString(str(row["review"]).lower()) + ',' + row["rating"]
    return convertedReviews

def convertReviewsLib(reviews):
    convertedReviews = []
    for a in reviews:
        convertedReviews.append(processString(str(a).lower()))
    return convertedReviews


In [8]:
# String is used to send each row for preprocessing
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

# Regex for non alphanumeric characters
regex_str = [
    r'<[^>]+>',  # HTML tags
    r"(?:[a-z][a-z\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

In [9]:
stopWords = []
decisionAttributes = []
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)
punctuation = list(string.punctuation)
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
initializeSystem()

Preparing stop words.


In [10]:
train_review1=train_review.map(lambda a: processString(str(a).lower())).collect()

max_df : float in range [0.0, 1.0] or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_df : float in range [0.0, 1.0] or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

max_features : int or None, default=None
If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

This parameter is ignored if vocabulary is not None.

In [11]:
# min_df=10,max_df=0.9,max_features=500
count_vectorizer = CountVectorizer(max_df = 0.7, max_features=500)
counts = count_vectorizer.fit_transform(train_review1)

In [12]:
dataFeatures=pd.DataFrame(counts.toarray(), columns=count_vectorizer.get_feature_names())

In [13]:
dataFeatures.shape

(59883, 500)

In [14]:
dataFeatures.head()

,10,15,20,30,abl,absolut,access,accommod,across,actual,...,wi,wifi,window,within,without,wonder,work,worth,would,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
clf = KNeighborsClassifier(n_neighbors=20, weights='distance', algorithm='brute')
clf.fit(dataFeatures, train_rating.collect())

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='distance')

In [16]:
test_review1 = test_review.map(lambda a: processString(str(a).lower())).collect()
testfeatures = count_vectorizer.transform(test_review1)

In [17]:
testDataFeatures = pd.DataFrame(testfeatures.toarray(), columns=count_vectorizer.get_feature_names())

In [18]:
sc.broadcast(clf)

In [19]:
testdf = sc.parallelize(testDataFeatures.values)

In [20]:
results = testdf.map(lambda x: clf.predict([x])[0]).collect()

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(test_rating.collect(), results)

0.308048956801356

In [24]:
def predic_rating(s):
    print(clf.predict(count_vectorizer.transform([s]).toarray())[0])

In [25]:
predic_rating('The beds were very good, The staff were very supportive')

9


In [26]:
predic_rating('smoke small poor late limit heat noise bad late service dark wonder temperatur')

9


In [27]:
predic_rating('awesome rooms great view complimentarty dinner')

10


In [30]:
print(results[:400])

[10, 6, 9, 10, 5, 7, 9, 10, 9, 9, 9, 9, 8, 9, 10, 8, 10, 8, 9, 9, 10, 7, 7, 9, 9, 7, 8, 9, 9, 9, 7, 9, 9, 10, 7, 10, 10, 10, 9, 8, 9, 10, 9, 9, 9, 8, 9, 7, 9, 9, 7, 10, 10, 5, 9, 9, 8, 10, 8, 9, 9, 5, 9, 10, 9, 5, 8, 9, 9, 10, 9, 8, 9, 9, 9, 10, 8, 9, 9, 5, 9, 8, 10, 5, 10, 8, 10, 8, 10, 7, 10, 9, 7, 8, 7, 7, 9, 9, 9, 10, 10, 8, 7, 10, 7, 7, 10, 9, 3, 6, 5, 9, 7, 9, 9, 9, 10, 7, 10, 10, 7, 8, 10, 7, 9, 10, 10, 7, 7, 9, 9, 7, 9, 7, 10, 7, 10, 7, 10, 7, 7, 7, 9, 8, 10, 10, 7, 7, 10, 7, 9, 10, 10, 10, 7, 7, 10, 7, 9, 10, 10, 10, 8, 8, 6, 10, 7, 6, 7, 9, 10, 10, 9, 8, 9, 7, 9, 7, 7, 8, 9, 5, 9, 7, 9, 8, 7, 8, 7, 10, 9, 7, 8, 10, 5, 9, 7, 7, 7, 10, 9, 7, 8, 7, 7, 8, 9, 10, 8, 9, 7, 7, 7, 10, 9, 10, 7, 7, 7, 7, 7, 9, 7, 9, 7, 10, 7, 7, 5, 9, 9, 9, 10, 7, 7, 5, 8, 10, 10, 9, 5, 10, 6, 5, 10, 10, 7, 10, 7, 9, 10, 10, 9, 10, 9, 10, 10, 9, 8, 8, 9, 10, 10, 9, 8, 9, 10, 10, 10, 10, 10, 8, 7, 7, 9, 7, 5, 10, 8, 10, 10, 8, 10, 9, 5, 7, 10, 9, 7, 7, 9, 7, 9, 10, 7, 9, 7, 9, 7, 8, 8, 10, 8, 10, 8, 10

In [29]:
print(count_vectorizer.get_feature_names())

[u'10', u'15', u'20', u'30', u'abl', u'absolut', u'access', u'accommod', u'across', u'actual', u'add', u'addit', u'air', u'airport', u'almost', u'also', u'although', u'alway', u'amaz', u'amen', u'amsterdam', u'anoth', u'anyth', u'apart', u'appreci', u'area', u'around', u'arriv', u'ask', u'atmospher', u'attent', u'attract', u'avail', u'away', u'back', u'bad', u'bag', u'balconi', u'bar', u'barcelona', u'basic', u'bath', u'bathroom', u'beauti', u'bed', u'bedroom', u'best', u'better', u'big', u'bigger', u'birthday', u'bite', u'block', u'book', u'bottl', u'break', u'breakfast', u'brilliant', u'bring', u'bu', u'buffet', u'build', u'busi', u'cafe', u'call', u'car', u'card', u'care', u'carpet', u'center', u'centr', u'central', u'chang', u'charg', u'check', u'choic', u'choos', u'citi', u'clean', u'cleanli', u'close', u'club', u'coffe', u'cold', u'com', u'come', u'comfi', u'comfort', u'compar', u'complain', u'complaint', u'complimentari', u'con', u'concierg', u'condit', u'connect', u'consid', u'